In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [99]:
from keras.models import Sequential
from keras.layers import Dense, Activation

from sklearn.model_selection import train_test_split


In [3]:
!ls

Untitled.ipynb         data                   keras_model1_v00.ipynb


In [4]:
!ls data/processed

sample_submission.csv test.json             train.json


In [5]:
print("load data...")
train = pd.read_json("data/processed/train.json")
test = pd.read_json("data/processed/test.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
print("done!")

load data...
done!


In [10]:
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
                          , x_band2[:, :, :, np.newaxis]
                         , ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)
X_angle_train = np.array(train.inc_angle)
y_train = np.array(train["is_iceberg"])





In [100]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [101]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((1074, 75, 75, 3), (530, 75, 75, 3), (1074,), (530,))

In [31]:
train["is_iceberg"].value_counts()

0    851
1    753
Name: is_iceberg, dtype: int64

In [33]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=5625))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [37]:
data = np.random.random((1000, 5625))
labels = np.random.randint(2, size=(1604, 1))

# Train the model, iterating on the data in batches of 32 samples
model.fit(x_band1.reshape( 1604,5625  ), y_train, epochs=10, batch_size=32)

Epoch 1/10
1604/1604 [==============================] - 0s - loss: 8.4582 - acc: 0.4695     
Epoch 2/10
1604/1604 [==============================] - 0s - loss: 8.4582 - acc: 0.4695     
Epoch 3/10
1604/1604 [==============================] - 0s - loss: 8.4582 - acc: 0.4695     
Epoch 4/10
1604/1604 [==============================] - 0s - loss: 8.4582 - acc: 0.4695     
Epoch 5/10
1604/1604 [==============================] - 0s - loss: 8.4582 - acc: 0.4695     
Epoch 6/10
1604/1604 [==============================] - 0s - loss: 8.4582 - acc: 0.4695     
Epoch 7/10
1604/1604 [==============================] - 0s - loss: 8.4582 - acc: 0.4695     
Epoch 8/10
1604/1604 [==============================] - 0s - loss: 8.4582 - acc: 0.4695     
Epoch 9/10
1604/1604 [==============================] - 0s - loss: 8.4582 - acc: 0.4695     
Epoch 10/10
1604/1604 [==============================] - 0s - loss: 8.4582 - acc: 0.4695     


In [95]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation, ELU, Add
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.preprocessing.image import ImageDataGenerator


In [104]:
def get_model():
    bn_model = 0.99
    p_activation = "elu"
    k_size = (5,5)
    input_1 = Input(shape=(75, 75, 3), name="X_1")
    input_2 = Input(shape=[1], name="angle")
    
    img_1 = Conv2D(32, kernel_size = k_size, activation=p_activation, padding="same") ((BatchNormalization(momentum=bn_model))(input_1))
    img_1 = Conv2D(32, kernel_size = k_size, activation=p_activation, padding="same") (img_1)
    img_1 = MaxPooling2D((2,2)) (img_1)
    img_1 = Dropout(0.25)(img_1)

    img_1 = Conv2D(64, kernel_size = k_size, activation=p_activation, padding="same")((BatchNormalization(momentum=bn_model))(img_1))
    img_1 = Conv2D(64, kernel_size = k_size, activation=p_activation, padding="same")(img_1)
    img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
    img_1 = Dropout(0.25)(img_1)    
    
    print("img1 shape", img_1.shape)

    img_resid = Conv2D(128, kernel_size = k_size, activation=p_activation, padding="same")((BatchNormalization(momentum=bn_model))(img_1))
    img_resid = Conv2D(128, kernel_size = k_size, activation=p_activation, padding="same")(img_resid)
    img_resid = Dropout(0.25)(img_resid)    
    print("img_resid shape1", img_resid.shape)
    
    img_resid = Conv2D(64, kernel_size = k_size, activation=p_activation, padding="same")((BatchNormalization(momentum=bn_model))(img_resid))
    img_resid = Conv2D(64, kernel_size = k_size, activation=p_activation, padding="same")(img_resid)
    print("img_resid shape2", img_resid.shape)

    cnn_resid_added = Add()([img_1, img_resid])
    print("cnn_resid shape", cnn_resid_added.shape)
    
    top_cnn = Conv2D(128, kernel_size = k_size, activation=p_activation, padding="same")((BatchNormalization(momentum=bn_model))(cnn_resid_added))
    top_cnn = Conv2D(128, kernel_size = k_size, activation=p_activation, padding="same")(top_cnn)
    top_cnn = MaxPooling2D((2,2)) (top_cnn)
    top_cnn = Conv2D(256, kernel_size = k_size, activation=p_activation, padding="same")((BatchNormalization(momentum=bn_model))(top_cnn))
    top_cnn = Conv2D(256, kernel_size = k_size, activation=p_activation, padding="same")(top_cnn)
    top_cnn = Dropout(0.25)(top_cnn)
    top_cnn = MaxPooling2D((2,2)) (top_cnn)
    top_cnn = Conv2D(512, kernel_size = k_size, activation=p_activation, padding="same")((BatchNormalization(momentum=bn_model))(top_cnn))
    top_cnn = Conv2D(512, kernel_size = k_size, activation=p_activation, padding="same")(top_cnn)
    top_cnn = Dropout(0.25)(top_cnn)
    top_cnn = MaxPooling2D((2,2)) (top_cnn)
    
    top_cnn = GlobalMaxPooling2D()(top_cnn)
    print("top_cnn shape", top_cnn.shape)

    dense_ayer = ELU()(BatchNormalization(momentum=bn_model)( Dense(512, activation=None)(top_cnn) ))
    dense_ayer = Dropout(0.5)(dense_ayer)
    dense_ayer = ELU()(BatchNormalization(momentum=bn_model)( Dense(256, activation=None)(dense_ayer) ))
    dense_ayer = Dropout(0.5)(dense_ayer)
    
    output = Dense(1, activation="softmax")(dense_ayer)
    
    
    
    model = Model([input_1],  output)
    optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model


In [105]:
model = get_model()
model.summary()

('img1 shape', TensorShape([Dimension(None), Dimension(18), Dimension(18), Dimension(64)]))
('img_resid shape1', TensorShape([Dimension(None), Dimension(18), Dimension(18), Dimension(128)]))
('img_resid shape2', TensorShape([Dimension(None), Dimension(18), Dimension(18), Dimension(64)]))
('cnn_resid shape', TensorShape([Dimension(None), Dimension(18), Dimension(18), Dimension(64)]))
('top_cnn shape', TensorShape([Dimension(None), Dimension(512)]))
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
X_1 (InputLayer)                 (None, 75, 75, 3)     0                                            
____________________________________________________________________________________________________
batch_normalization_142 (BatchNo (None, 75, 75, 3)     12          X_1[0][0]                        
___________________________________________

# training parameters #

In [92]:
batch_size = 64
epochs = 5 ## change this to 80
steps_per_epoch=2^9/batch_size ## change to 2^14

In [93]:
def get_callbacks(filepath, patience=10):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

In [106]:
gen_images = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.3,
        height_shift_range=0.3,
        #rescale=1./255,
        shear_range=0.2,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,)
        #fill_mode='nearest')

model.fit_generator(
    
        gen_images.flow(X_train,y_train,batch_size=batch_size),
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=(X_valid,y_valid),
        validation_steps=steps_per_epoch)

Epoch 1/5
2/2 [==============================] - 90s - loss: 8.9676 - acc: 0.4375 - val_loss: 8.4525 - val_acc: 0.4698
Epoch 2/5
2/2 [==============================] - 71s - loss: 8.4694 - acc: 0.4688 - val_loss: 8.4525 - val_acc: 0.4698
Epoch 3/5
2/2 [==============================] - 69s - loss: 8.8430 - acc: 0.4453 - val_loss: 8.4525 - val_acc: 0.4698
Epoch 4/5
2/2 [==============================] - 68s - loss: 8.8430 - acc: 0.4453 - val_loss: 8.4525 - val_acc: 0.4698
Epoch 5/5
2/2 [==============================] - 69s - loss: 7.8466 - acc: 0.5078 - val_loss: 8.4525 - val_acc: 0.4698


In [ ]:
gen_images <- image_data_generator(
  # featurewise_center = TRUE,
  # featurewise_std_normalization = TRUE,
  horizontal_flip = TRUE,
  vertical_flip = TRUE,
  width_shift_range = 0.30,
  height_shift_range = 0.30,
  zoom_range = 0.1,
  rotation_range = 20
  )


history = model %>% fit_generator(
  flow_images_from_data(x = train[[1]], y = train[[3]],generator = gen_images,
     batch_size=batch_size, seed = 123),
     steps_per_epoch=steps_per_epoch,epochs = epochs,
  view_metrics = T,
  validation_data = list(val[[1]],val[[3]]),
  callbacks = c(check_point,early_stopping)
)

In [ ]:


input_CNN = input_img_norm %>%
    layer_conv_2d(32, kernel_size = kernel_size,padding = "same") %>%
    layer_batch_normalization(momentum = 0.99) %>%
    layer_activation_elu() %>%
    layer_max_pooling_2d(c(2,2)) %>%
    layer_dropout(0.25) %>%
    layer_conv_2d(64, kernel_size = kernel_size,padding = "same") %>%
    layer_batch_normalization(momentum = 0.99) %>%
    layer_activation_elu() %>%
    layer_max_pooling_2d(c(2,2)) %>%
    layer_dropout(0.25) 

## first residual
input_CNN_residual = input_CNN %>%
  layer_batch_normalization(momentum = 0.99) %>%
  layer_conv_2d(128, kernel_size = kernel_size,padding = "same") %>%
  layer_batch_normalization(momentum = 0.99) %>%
  layer_activation_elu() %>%
  layer_dropout(0.25) %>%
  layer_conv_2d(64, kernel_size = kernel_size,padding = "same") %>%
  layer_batch_normalization(momentum = 0.99) %>%
  layer_activation_elu()

input_CNN_residual = layer_add(list(input_CNN_residual,input_CNN))

top_CNN = input_CNN_residual %>%
  layer_conv_2d(128, kernel_size = kernel_size,padding = "same") %>%
  layer_batch_normalization(momentum = 0.99) %>%
  layer_activation_elu() %>%
  layer_max_pooling_2d(c(2,2)) %>%
  layer_conv_2d(256, kernel_size = kernel_size,padding = "same") %>%
  layer_batch_normalization(momentum = 0.99) %>%
  layer_activation_elu() %>%
  layer_dropout(0.25) %>%
  layer_max_pooling_2d(c(2,2)) %>%
  layer_conv_2d(512, kernel_size = kernel_size,padding = "same") %>%
  layer_batch_normalization(momentum = 0.99) %>%
  layer_activation_elu() %>%
  layer_dropout(0.25) %>%
  layer_max_pooling_2d(c(2,2)) %>%
  layer_global_max_pooling_2d()

outputs = top_CNN %>%
    layer_dense(512,activation = NULL) %>%
    layer_batch_normalization(momentum = 0.99) %>%
    layer_activation_elu() %>%
    layer_dropout(0.5) %>%
    layer_dense(256,activation = NULL) %>%
    layer_batch_normalization(momentum = 0.99) %>%
    layer_activation_elu() %>%
    layer_dropout(0.5) %>%
    layer_dense(2,activation = "softmax") ## not sure using softmax is the right thing to do...
    
model <- keras_model(inputs = list(input_img), outputs = list(outputs))

model %>% compile(optimizer=optimizer_adam(lr = 0.001),
                  loss="binary_crossentropy",
                  metrics = c("accuracy"))

summary(model)